In [1]:
import tensorflow as tf
import numpy as np

In [2]:
num_arr = ['1','2','3','4','5','6','7','8','9','0']
num_dic = {n : i for i,n in enumerate(num_arr)}
dic_len = len(num_dic)

In [4]:
seq_data = ['1234','2345','3456','4567','5678','6789','7890']
seq_data2 = ['123','234','345','456','567','678','789','890']

In [5]:
def one_hot_seq(seq_data):
    x_batch = []
    y_batch = []
    for seq in seq_data:
        x_data = [num_dic[n] for n in seq[:-1]]
        y_data = num_dic[seq[-1]]
        x_batch.append(np.eye(dic_len)[x_data])
        y_batch.append([y_data])
        
    return x_batch, y_batch

In [11]:
n_input = 10
n_classes = 10
n_hidden = 128
n_layers = 3

In [12]:
X = tf.placeholder(tf.float32, [None,None, n_input])
Y = tf.placeholder(tf.int32, [None,1])
W = tf.Variable(tf.random_normal([n_hidden, n_classes]))
b = tf.Variable(tf.random_normal([n_classes]))

In [13]:
X_t = tf.transpose(X, [1,0,2])

[batch_size, n_steps, n_input] -> [n_steps, batch_size, n_input]

In [14]:
cell = tf.contrib.rnn.BasicRNNCell(n_hidden)
cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=0.5)
cell = tf.contrib.rnn.MultiRNNCell([cell]*n_layers)

In [15]:
outputs, states = tf.nn.dynamic_rnn(cell,X_t,dtype=tf.float32,time_major=True)

In [16]:
logits = tf.matmul(outputs[-1],W)+b

In [17]:
labels = tf.reshape(Y,[-1])

In [18]:
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,labels=labels))

In [19]:
train_op = tf.train.RMSPropOptimizer(learning_rate=0.01).minimize(cost)

In [20]:
#########
# 결과 확인
######
# 테스트 데이터를 받아 결과를 예측해보는 함수
def prediction(seq_data):
    prediction = tf.cast(tf.argmax(logits, 1), tf.int32)
    prediction_check = tf.equal(prediction, labels)
    accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

    x_batch_t, y_batch_t = one_hot_seq(seq_data)
    real, predict, accuracy_val = sess.run([labels, prediction, accuracy],
                                           feed_dict={X: x_batch_t, Y: y_batch_t})

    print("\n=== 예측 결과 ===")
    print('순차열:', seq_data)
    print('실제값:', [num_arr[i] for i in real])
    print('예측값:', [num_arr[i] for i in predict])
    print('정확도:', accuracy_val)

In [24]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    x_batch, y_batch = one_hot_seq(seq_data)
    x_batch2, y_batch2 = one_hot_seq(seq_data2)
    
    for epoch in range(30):
        _, loss4 = sess.run([train_op, cost], feed_dict={X:x_batch,
                                                         Y: y_batch})
        _, loss3 = sess.run([train_op, cost], feed_dict={X:x_batch2, Y:y_batch2})
        
        print('Epoch:', '%04d' % (epoch+1), 'cost =',\
             'bucket[4] =', '{:.6f}'.format(loss4),\
             'bucket[3] =', '{:.6f}'.format(loss3))
        
    # 학습 데이터에 있던 시퀀스로 테스트
    seq_data_test = ['123', '345', '789']
    prediction(seq_data_test)

    seq_data_test = ['1234', '2345', '7890']
    prediction(seq_data_test)

    # 학습시키지 않았던 시퀀스를 테스트 해 봅니다.
    seq_data_test = ['23', '78', '90']
    prediction(seq_data_test)

    seq_data_test = ['12345', '34567', '67890']
    prediction(seq_data_test)

Epoch: 0001 cost = bucket[4] = 7.992703 bucket[3] = 4.474046
Epoch: 0002 cost = bucket[4] = 4.535444 bucket[3] = 4.535411
Epoch: 0003 cost = bucket[4] = 1.731096 bucket[3] = 2.134820
Epoch: 0004 cost = bucket[4] = 2.357528 bucket[3] = 2.389493
Epoch: 0005 cost = bucket[4] = 1.016439 bucket[3] = 1.696504
Epoch: 0006 cost = bucket[4] = 0.842209 bucket[3] = 0.043311
Epoch: 0007 cost = bucket[4] = 0.453393 bucket[3] = 0.242600
Epoch: 0008 cost = bucket[4] = 0.079169 bucket[3] = 0.369865
Epoch: 0009 cost = bucket[4] = 0.738299 bucket[3] = 0.894375
Epoch: 0010 cost = bucket[4] = 0.788772 bucket[3] = 0.542474
Epoch: 0011 cost = bucket[4] = 0.184362 bucket[3] = 0.573481
Epoch: 0012 cost = bucket[4] = 0.089432 bucket[3] = 0.238450
Epoch: 0013 cost = bucket[4] = 0.003757 bucket[3] = 0.119641
Epoch: 0014 cost = bucket[4] = 0.346242 bucket[3] = 1.164445
Epoch: 0015 cost = bucket[4] = 0.022629 bucket[3] = 0.000396
Epoch: 0016 cost = bucket[4] = 0.008909 bucket[3] = 0.545011
Epoch: 0017 cost = bucke